In [13]:
import nltk
import numpy as np
import random
import lxml


import bs4 as bs
import urllib.request
import re
import ssl

In [16]:
from github import Github

# Replace <github_username> and <github_token> with your own username and personal access token
g = Github("<github_username>", "<github_token>")

# Replace <repo_owner> and <repo_name> with the owner and name of the repository you want to download
repo = g.get_repo("<repo_owner>/<repo_name>")

# Replace <download_path> with the path where you want to download the repository
download_path = "<download_path>"

# Download the repository to the specified path
repo.get_archive("zipball", download_path)

ImportError: cannot import name 'Github' from 'github' (/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/github/__init__.py)

In [ ]:
# Define the architecture of the neural network
input_size = 2
hidden_size = 3
output_size = 1

# Initialize the weights and biases
W1 = np.random.randn(hidden_size, input_size) * 0.01
b1 = np.zeros((hidden_size, 1))
W2 = np.random.randn(output_size, hidden_size) * 0.01
b2 = np.zeros((output_size, 1))

# Define the activation function (sigmoid)
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

# Implement the forward propagation algorithm
def forward(X):
    Z1 = np.dot(W1, X) + b1
    A1 = sigmoid(Z1)
    Z2 = np.dot(W2, A1) + b2
    A2 = sigmoid(Z2)
    return A2

# Implement the backward propagation algorithm
def backward(X, Y, A2):
    dZ2 = A2 - Y
    dW2 = np.dot(dZ2, A1.T)
    db2 = np.sum(dZ2, axis=1, keepdims=True)
    dZ1 = np.dot(W2.T, dZ2) * A1 * (1 - A1)
    dW1 = np.dot(dZ1, X.T)
    db1 = np.sum(dZ1, axis=1, keepdims=True)
    return dW1, db1, dW2, db2

# Train the neural network using stochastic gradient descent
X = np.array([[0, 0], [0, 1], [1, 0], [1, 1]]).T
Y = np.array([[0, 1, 1, 0]])
learning_rate = 0.1
num_iterations = 10000
for i in range(num_iterations):
    A2 = forward(X)
    dW1, db1, dW2, db2 = backward(X, Y, A2)
    W1 -= learning_rate * dW1
    b1 -= learning_rate * db1
    W2 -= learning_rate * dW2
    b2 -= learning_rate * db2

# Test the neural network
predictions = forward(X)
print(predictions)
